In [2]:
import sys
import pygame
import os
from time import sleep

a = os.path.lexists("/Users/Desktop/Documents/PythonFundamentos/Python_crash_course/alien_invasion/images/ship.bmp")  



from settings import Settings
from game_stats import GameStats
from scoreboard import Scoreboard
from button import Button
from ship import Ship
from alien import Alien
from bullet import Bullet


class AlienInvasion:
    """Overall class to manage game assets and behavior."""



    def __init__(self):
        """Initialize the game, and create game resources."""
        pygame.init() #this function initializes the background settings that
                  #Pygame needs to work properly
        self.settings = Settings()
        self.screen = pygame.display.set_mode((self.settings.screen_width,self.settings.screen_height))
    
        self.ship = Ship(self)
        
        #Create a instance to store game statistics.
        self.stats = GameStats(self)
        self.sb = Scoreboard(self)
        self.bullets = pygame.sprite.Group()
        self.aliens = pygame.sprite.Group()
        
        self._create_fleet()


        #self.screen = pygame.display.set_mode((1200,800)) #pygame.display.set_mode create a display window, 
                                                      #on which we’ll draw all the game’s graphical elements
                                                      #The argument (1200, 800) is a tuple that defines the dimensions of
                                                      #the game window, which will be 1200 pixels wide by 800 pixels high. (You
                                                      #can adjust these values depending on your display size.)
        #Make the play button.
        self.play_button = Button(self,"Play")
        
        
        
        pygame.display.set_caption("Alien Invasion")
        
        

   
    

    def run_game(self):
        """Star the main look for the game"""
        while True:
            self._check_events()
            if self.stats.game_active:
                
                self.ship.update()
                self._update_bullets()
                self._update_aliens()
                
            self._update_screen()
            
            
            
    
    def _check_events(self):
        #Responding for keypresses and mouse events
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                    
                
                elif event.type == pygame.KEYDOWN:
                    self._check_keydown_events(event)
                elif event.type == pygame.KEYUP:
                    self._check_keyup_events(event)
                elif event.type ==pygame.MOUSEBUTTONDOWN:
                    mouse_pos = pygame.mouse.get_pos()
                    self._check_play_button(mouse_pos)
                    
                    
                    
    def _check_play_button(self,mouse_pos):
        """ Start a new game then player clicks Play."""
        button_clicked = self.play_button.rect.collidepoint(mouse_pos)
        if button_clicked and not self.stats.game_active:
            #Reset the game settings.
            self.settings.initialize_dynamic_settings()
            
        
            #Reset the game statistics.
            self.stats.reset_stats()
            self.stats.game_active = True
            self.sb.prep_score()
            self.sb.prep_level()
            self.sb.prep_ships()
            
            #Get rid of any remaining aliens and bullets.
            self.aliens.empty()
            self.bullets.empty()
            
            #Create a new fleet and center the ship.
            self._create_fleet()
            self.ship.center_ship()
            
            #Hide the mouse cursor.
            pygame.mouse.set_visible(False)
                    
                    
    def _check_keydown_events(self,event):
        """Respond to keypresses."""
        if event.key == pygame.K_RIGHT:
            #move the ship to the right.
            self.ship.moving_right = True
        elif event.key == pygame.K_LEFT:
            #move the ship to the right.
            self.ship.moving_left = True
        elif event.key == pygame.K_q:
            pygame.quit()
            sys.exit()
        elif event.key == pygame.K_SPACE:
            self._fire_bullet()
                        
                        
    def _check_keyup_events(self,event):
        """Respond to key releases"""
        if event.key == pygame.K_RIGHT:
            self.ship.moving_right = False
        elif event.key == pygame.K_LEFT:
            self.ship.moving_left = False
                    
    def _fire_bullet(self):
        """Create a new bullet and add it to the bullets group."""
        if len(self.bullets)<self.settings.bullets_allowed:
            new_bullet = Bullet(self)
            self.bullets.add(new_bullet)
        
        
                       

    def _update_screen(self):
        """Update Images on the screen, and flip to the new screen."""
        #Redraw the screen during each pass through the loop.
      
        self.screen.fill(self.settings.bg_color)
        self.ship.blitme()
        for bullet in self.bullets.sprites():
            bullet.draw_bullet()
      
        #Make the most recently drawn screen visible.
        self.aliens.draw(self.screen)
        
        #Draw the score information.
        self.sb.show_score()
        
        #Draw the play button if the game is inactive.
        if not self.stats.game_active:
            self.play_button.draw_button()
        
        
        
        
        pygame.display.flip()
        
    def _update_bullets(self):
        """Update position of bullets an get rid of old bullets."""
        #Update bullet positions.
        self.bullets.update()
        
        #Get rid of bullets that have disappeared.
        for bullet in self.bullets.copy():
            if bullet.rect.bottom <=0:
                self.bullets.remove(bullet)
        #print(len(self.bullets)
        self._check_bullet_alien_collisions()
        
    def _check_bullet_alien_collisions(self):
        """Respond to bullet-alien collisions."""
        #Remove any bullets and alians that have collided.
        
        #Check for any bullets that have hit aliens.
        # If so, get rid of the bullet and the aliens.
        
        collisions = pygame.sprite.groupcollide(self.bullets, self.aliens, 
                                                True,True)
        
        if collisions:
            for aliens in collisions.values():
                self.stats.score +=self.settings.alien_points *len(aliens)
            self.sb.prep_score()
            self.sb.check_high_score()
        
        
        
        if not self.aliens:
            #Destroy existing bullets and create new fleet.
            self.bullets.empty()
            self._create_fleet()
            self.settings.increase_speed()
            
            #Increse level.
            self.stats.level += 1
            self.sb.prep_level()
            
        
        
    
        
        
    def _create_fleet(self):
        """Create the fleet of aliens."""
        #Make an alien.
        #Create an alien and find the number of aliens in a row.
        #Spacing between each alien is equal to one alien width.
        alien = Alien(self)
        alien_width,alien_height = alien.rect.size
        available_space_x = self.settings.screen_width - (2 * alien_width)
        number_aliens_x = available_space_x // (2 * alien_width)
        
        #Determine the number of rows of aliens that fit ont he screen.
        ship_height = self.ship.rect.height
        available_space_y = (self.settings.screen_height - (3 * alien_height) - ship_height)
        numbers_rows = available_space_y // (2  * alien_height)
        
        #Create the full fleet of aliens.
        for row_number in range(numbers_rows):
            for alien_number in range(number_aliens_x):
                self._create_alien(alien_number,row_number)
            
    def _create_alien(self,alien_number,row_number):
        """Create an alien and place it in the row."""
        alien = Alien(self)
        alien_width, alien_height = alien.rect.size
        alien.x = alien_width + 2 *alien_width * alien_number
        alien.rect.x = alien.x
        alien.rect.y = alien.rect.height + 2 * alien.rect.height * row_number
        self.aliens.add(alien)
        
        
    def _check_fleet_edges(self):
        """Respond appropriately if any aliens have reached an edge."""
        for alien in self.aliens.sprites():
            if alien.check_edges():
                self._change_fleet_direction()
                break
                
    def _change_fleet_direction(self):
        """Drop the entire fleet and change the fleet's direction."""
        for alien in self.aliens.sprites():
            alien.rect.y += self.settings.fleet_drop_speed
        self.settings.fleet_direction *= -1
        
    def _update_aliens(self):
        """
        Check if the fleet is at an edge,
          then update the positions of all aliens in the fleet.
        """
        self._check_fleet_edges()
        self.aliens.update()
        
        #Look for alien-ship collisions.
        if pygame.sprite.spritecollideany(self.ship,self.aliens):
            #print("Ship hit!!!")
            self._ship_hit()
            
        #Look for aliens hitting the bottom of of the screen.
        self._check_aliens_bottom()
            
            
    def _ship_hit(self):
        """Respond to the ship being hit by an alien."""
        if self.stats.ships_left > 0:
            #Decrement ships_left and update scoreboard.
            self.stats.ships_left -=1
            self.sb.prep_ships()
        
            #Get rid of any remaining aliens and bullets.
            self.aliens.empty()
            self.bullets.empty()
        
        
            #Create a new fleet and center the ship.
            self._create_fleet()
            self.ship.center_ship()
        
            #Pause.
            sleep(0.5)
        else:
            self.stats.game_active = False
            pygame.mouse.set_visible(True)
        
        
        
    def _check_aliens_bottom(self):
        """Check if any alien have reached the bottom fo the screen."""
        screen_rect = self.screen.get_rect()
        for alien in self.aliens.sprites():
            if alien.rect.bottom >= screen_rect.bottom:
                #Treat this the same as if the ship got hit.
                self._ship_hit()
                break
    
        
        
        
    

if __name__ == '__main__':
    #Make a game instance, and run the game.
    ai_game = AlienInvasion()
    ai_game.run_game()


SystemExit: 

In [2]:
import sys
import pygame
import os
from time import sleep

a = os.path.lexists("/Users/Desktop/Documents/PythonFundamentos/Python_crash_course/alien_invasion/images/ship.bmp")  



from settings import Settings
from game_stats import GameStats
from scoreboard import Scoreboard
from button import Button
from ship import Ship
from alien import Alien
from bullet import Bullet


class AlienInvasion:
    """Overall class to manage game assets and behavior."""



    def __init__(self):
        """Initialize the game, and create game resources."""
        pygame.init() #this function initializes the background settings that
                  #Pygame needs to work properly
        self.settings = Settings()
        self.screen = pygame.display.set_mode((self.settings.screen_width,self.settings.screen_height))
    
        self.ship = Ship(self)
        
        #Create a instance to store game statistics.
        self.stats = GameStats(self)
        self.sb = Scoreboard(self)
        self.bullets = pygame.sprite.Group()
        self.aliens = pygame.sprite.Group()
        
        self._create_fleet()


        #self.screen = pygame.display.set_mode((1200,800)) #pygame.display.set_mode create a display window, 
                                                      #on which we’ll draw all the game’s graphical elements
                                                      #The argument (1200, 800) is a tuple that defines the dimensions of
                                                      #the game window, which will be 1200 pixels wide by 800 pixels high. (You
                                                      #can adjust these values depending on your display size.)
        #Make the play button.
        self.play_button = Button(self,"Play")
        
        
        
        pygame.display.set_caption("Alien Invasion")
        
        

   
    

    def run_game(self):
        """Star the main look for the game"""
        while True:
            self._check_events()
            if self.stats.game_active:
                
                self.ship.update()
                self._update_bullets()
                self._update_aliens()
                
            self._update_screen()
            
            
            
    
    def _check_events(self):
        #Responding for keypresses and mouse events
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                    
                
                elif event.type == pygame.KEYDOWN:
                    self._check_keydown_events(event)
                elif event.type == pygame.KEYUP:
                    self._check_keyup_events(event)
                elif event.type ==pygame.MOUSEBUTTONDOWN:
                    mouse_pos = pygame.mouse.get_pos()
                    self._check_play_button(mouse_pos)
                    
                    
                    
    def _check_play_button(self,mouse_pos):
        """ Start a new game then player clicks Play."""
        button_clicked = self.play_button.rect.collidepoint(mouse_pos)
        if button_clicked and not self.stats.game_active:
            #Reset the game settings.
            self.settings.initialize_dynamic_settings()
            
        
            #Reset the game statistics.
            self.stats.reset_stats()
            self.stats.game_active = True
            self.sb.prep_score()
            self.sb.prep_level()
            self.sb.prep_ships()
            
            #Get rid of any remaining aliens and bullets.
            self.aliens.empty()
            self.bullets.empty()
            
            #Create a new fleet and center the ship.
            self._create_fleet()
            self.ship.center_ship()
            
            #Hide the mouse cursor.
            pygame.mouse.set_visible(False)
                    
                    
    def _check_keydown_events(self,event):
        """Respond to keypresses."""
        if event.key == pygame.K_RIGHT:
            #move the ship to the right.
            self.ship.moving_right = True
        elif event.key == pygame.K_LEFT:
            #move the ship to the right.
            self.ship.moving_left = True
        elif event.key == pygame.K_q:
            pygame.quit()
            sys.exit()
        elif event.key == pygame.K_SPACE:
            self._fire_bullet()
                        
                        
    def _check_keyup_events(self,event):
        """Respond to key releases"""
        if event.key == pygame.K_RIGHT:
            self.ship.moving_right = False
        elif event.key == pygame.K_LEFT:
            self.ship.moving_left = False
                    
    def _fire_bullet(self):
        """Create a new bullet and add it to the bullets group."""
        if len(self.bullets)<self.settings.bullets_allowed:
            new_bullet = Bullet(self)
            self.bullets.add(new_bullet)
        
        
                       

    def _update_screen(self):
        """Update Images on the screen, and flip to the new screen."""
        #Redraw the screen during each pass through the loop.
      
        self.screen.fill(self.settings.bg_color)
        self.ship.blitme()
        for bullet in self.bullets.sprites():
            bullet.draw_bullet()
      
        #Make the most recently drawn screen visible.
        self.aliens.draw(self.screen)
        
        #Draw the score information.
        self.sb.show_score()
        
        #Draw the play button if the game is inactive.
        if not self.stats.game_active:
            self.play_button.draw_button()
        
        
        
        
        pygame.display.flip()
        
    def _update_bullets(self):
        """Update position of bullets an get rid of old bullets."""
        #Update bullet positions.
        self.bullets.update()
        
        #Get rid of bullets that have disappeared.
        for bullet in self.bullets.copy():
            if bullet.rect.bottom <=0:
                self.bullets.remove(bullet)
        #print(len(self.bullets)
        self._check_bullet_alien_collisions()
        
    def _check_bullet_alien_collisions(self):
        """Respond to bullet-alien collisions."""
        #Remove any bullets and alians that have collided.
        
        #Check for any bullets that have hit aliens.
        # If so, get rid of the bullet and the aliens.
        
        collisions = pygame.sprite.groupcollide(self.bullets, self.aliens, 
                                                True,True)
        
        if collisions:
            for aliens in collisions.values():
                self.stats.score +=self.settings.alien_points *len(aliens)
            self.sb.prep_score()
            self.sb.check_high_score()
        
        
        
        if not self.aliens:
            #Destroy existing bullets and create new fleet.
            self.bullets.empty()
            self._create_fleet()
            self.settings.increase_speed()
            
            #Increse level.
            self.stats.level += 1
            self.sb.prep_level()
            
        
        
    
        
        
    def _create_fleet(self):
        """Create the fleet of aliens."""
        #Make an alien.
        #Create an alien and find the number of aliens in a row.
        #Spacing between each alien is equal to one alien width.
        alien = Alien(self)
        alien_width,alien_height = alien.rect.size
        available_space_x = self.settings.screen_width - (2 * alien_width)
        number_aliens_x = available_space_x // (2 * alien_width)
        
        #Determine the number of rows of aliens that fit ont he screen.
        ship_height = self.ship.rect.height
        available_space_y = (self.settings.screen_height - (3 * alien_height) - ship_height)
        numbers_rows = available_space_y // (2  * alien_height)
        
        #Create the full fleet of aliens.
        for row_number in range(numbers_rows):
            for alien_number in range(number_aliens_x):
                self._create_alien(alien_number,row_number)
            
    def _create_alien(self,alien_number,row_number):
        """Create an alien and place it in the row."""
        alien = Alien(self)
        alien_width, alien_height = alien.rect.size
        alien.x = alien_width + 2 *alien_width * alien_number
        alien.rect.x = alien.x
        alien.rect.y = alien.rect.height + 2 * alien.rect.height * row_number
        self.aliens.add(alien)
        
        
    def _check_fleet_edges(self):
        """Respond appropriately if any aliens have reached an edge."""
        for alien in self.aliens.sprites():
            if alien.check_edges():
                self._change_fleet_direction()
                break
                
    def _change_fleet_direction(self):
        """Drop the entire fleet and change the fleet's direction."""
        for alien in self.aliens.sprites():
            alien.rect.y += self.settings.fleet_drop_speed
        self.settings.fleet_direction *= -1
        
    def _update_aliens(self):
        """
        Check if the fleet is at an edge,
          then update the positions of all aliens in the fleet.
        """
        self._check_fleet_edges()
        self.aliens.update()
        
        #Look for alien-ship collisions.
        if pygame.sprite.spritecollideany(self.ship,self.aliens):
            #print("Ship hit!!!")
            self._ship_hit()
            
        #Look for aliens hitting the bottom of of the screen.
        self._check_aliens_bottom()
            
            
    def _ship_hit(self):
        """Respond to the ship being hit by an alien."""
        if self.stats.ships_left > 0:
            #Decrement ships_left and update scoreboard.
            self.stats.ships_left -=1
            self.sb.prep_ships()
        
            #Get rid of any remaining aliens and bullets.
            self.aliens.empty()
            self.bullets.empty()
        
        
            #Create a new fleet and center the ship.
            self._create_fleet()
            self.ship.center_ship()
        
            #Pause.
            sleep(0.5)
        else:
            self.stats.game_active = False
            pygame.mouse.set_visible(True)
        
        
        
    def _check_aliens_bottom(self):
        """Check if any alien have reached the bottom fo the screen."""
        screen_rect = self.screen.get_rect()
        for alien in self.aliens.sprites():
            if alien.rect.bottom >= screen_rect.bottom:
                #Treat this the same as if the ship got hit.
                self._ship_hit()
                break
    
        
        
        
    

if __name__ == '__main__':
    #Make a game instance, and run the game.
    ai_game = AlienInvasion()
    ai_game.run_game()


SystemExit: 